# **<center>FAISS Tutorial</center>**

In [1]:
# #Install Packages
# !pip install faiss-cpu
# !pip install sentence-transformers

In [2]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [5]:
df = pd.read_csv("../data/sample_text.csv")
print(df.shape)
df

(8, 2)


,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


#### **Step 1 : Create source embeddings for the text column**

In [7]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)
print(vectors.shape)
dim = vectors.shape[1]
print(dim)

d:\Personal Files\Projects\Equity_Research_ChatBot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 392kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 63.3kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 2.60MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 37.5kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 202kB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [01:13<00:00, 5.94MB/s] 
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<?, ?B/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<00:00, 79.7kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 780kB/s]
tokenizer_config.json: 100%|██████████| 363/363 [00:00<00:00, 121kB/s]
train_scrip

(8, 768)
768


#### **Step 2 : Build a FAISS Index for vectors**

In [8]:
import faiss
index = faiss.IndexFlatL2(dim)

#### **Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index**

In [9]:
index.add(vectors)
print(index)

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000002AE35A17CC0> >


#### **Step 4 : Encode search text using same encorder and normalize the output vector**

In [10]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
print(vec.shape)

import numpy as np
svec = np.array(vec).reshape(1,-1)
print(svec.shape)

# faiss.normalize_L2(svec)

(768,)
(1, 768)


#### **Step 5: Search for similar vector in the FAISS index created**

In [15]:
distances, I = index.search(svec, k=2)
print(distances)
print(I)
print(I.tolist())
row_indices = I.tolist()[0]
print(row_indices)


[[1.384484  1.4039098]]
[[3 2]]
[[3, 2]]
[3, 2]


In [16]:
print(search_query)
df.loc[row_indices]

I want to buy a polo t-shirt


,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


- You can see that the two results from the dataframe are similar to a search_query

# **<center>THE END</center>**